In [16]:
import tensorflow as tf
import scipy as sp
from scipy import ndimage
import numpy as np
import os
import sklearn
from sklearn import preprocessing
from matplotlib import pyplot as plt
%matplotlib inline

In [18]:
# The inputs and outptus are the maps downloaded from https://www.nnvl.noaa.gov/view/globaldata.html
# The inputs to the model are Land Surface Temperature, Moisture and Longwave Energy. 
# The output is the average precipitation map. 
# All maps are in the average weekly format. The idea is to find correlation between the input maps and 
# the output. 

input_labels = ['fire', 'frac', 'land', 'moisture', 'ndvi', 'longwave_energy'];
files = ['./fire_data.txt', './frac_data.txt', './land_data.txt', './mois_data.txt', './ndvi_data.txt', './olwr_data.txt']
input_dict = {}

xdim = 256
ydim = 512
in_chan = 6
out_chan = 1

# Process the input : input_metrics will contain a 2048 x 4096 image with 9 channels (corresponding to 3 inputs)

for label, path in zip(input_labels, files):
    
    input_dict[label] = [];
    input_dict[label] = (np.loadtxt(path,delimiter=',').reshape(-1,xdim,ydim,1))
    print(label, path,input_dict[label].shape)

input_metrics = input_dict[input_labels[0]];
for label in (input_labels[1:]):
    input_metrics = np.concatenate((input_metrics, input_dict[label]), axis=3);

# Process the output : output_metric will contain a 256 x 4096 image with 3 channels
path = './rain_data.txt'
output_metric = (np.loadtxt(path, delimiter=',').reshape(-1,xdim,ydim,1))
output_metric = np.array(output_metric);
samples = input_metrics.shape[0];

print(input_metrics.shape)
print(output_metric.shape)


fire ./fire_data.txt (89, 256, 512, 1)
frac ./frac_data.txt (89, 256, 512, 1)
land ./land_data.txt (89, 256, 512, 1)
moisture ./mois_data.txt (89, 256, 512, 1)
ndvi ./ndvi_data.txt (89, 256, 512, 1)
longwave_energy ./olwr_data.txt (89, 256, 512, 1)
(89, 256, 512, 6)
(89, 256, 512, 1)


In [23]:
# Normalize data along the images and the inputs for all channels.

for i in range(input_metrics.shape[0]):
    for c in range(in_chan):
        input_metrics[i,:,:,c] = preprocessing.normalize(input_metrics[i,:,:,c])

    output_metric[i,:,:,0] = preprocessing.normalize(output_metric[i,:,:,0]);

In [24]:
# Function to obtain the next batch based on the input size and the batch size
def next_batch(indices, i):
    
    ind0, ind1 = i*batch_size, np.minimum((i+1)*batch_size, samples)
            
    return input_metrics[indices[ind0:ind1], :, :, :], output_metric[indices[ind0:ind1], :, :, :]

In [25]:
# Building a simple CNN model that looks like an auto-encoder. This is the section to change for a new model.

def conv_net(x):
    
    x = tf.reshape(x, shape=[-1, xdim, ydim, in_chan], name='reshape_x');
    x = tf.cast(x, tf.float32) 
    
    # Encoder
    # Scale down by 2x2. Out Channels = 32
    conv1 = tf.nn.relu(tf.contrib.layers.conv2d(x, 32, [8, 8], stride=2, padding='SAME', 
                                                biases_initializer=tf.zeros_initializer()))
    # Scale down by 2x2. Out Channels = 16
    conv2 = tf.nn.relu(tf.contrib.layers.conv2d(conv1, 16, [8, 8], stride=2, padding='SAME', 
                                                biases_initializer=tf.zeros_initializer()))
    # Scale down by 2x2. Out Channels = 8
    conv3 = tf.nn.relu(tf.contrib.layers.conv2d(conv2, 8, [8, 8], stride=2, padding='SAME', 
                                                biases_initializer=tf.zeros_initializer()))
    
    # Decoder
    # Scale up by 2x2. Out Channels = 16
    conv4 = tf.nn.relu(tf.contrib.layers.conv2d_transpose(conv3, 16, [8, 8], stride=2, padding='SAME',
                                                         biases_initializer=tf.zeros_initializer()))
    # Scale up by 2x2. Out Channels = 32
    conv5 = tf.nn.relu(tf.contrib.layers.conv2d_transpose(conv4, 32, [8, 8], stride=2, padding='SAME',
                                                         biases_initializer=tf.zeros_initializer()))
    # Scale up by 2x2. Out Channels = 1
    conv6 = tf.nn.relu(tf.contrib.layers.conv2d_transpose(conv5, out_chan, [8, 8], stride=2, padding='SAME',
                                                         biases_initializer=tf.zeros_initializer()))
    
    return conv6;

Y = tf.placeholder(tf.float32, shape=(None,xdim,ydim,out_chan));
X = tf.placeholder(tf.float32, shape=[None,xdim,ydim,in_chan]);

optimizer = tf.train.AdamOptimizer(learning_rate=0.001);

out = conv_net(X);
loss_op = tf.reduce_sum(tf.multiply(Y-out,Y-out));
train_op = optimizer.minimize(loss_op);

In [26]:
num_epochs = 200;
batch_size = 8;
num_batches = int(np.ceil(float(samples) / batch_size));
print(num_batches)

loss_arr = [];

with tf.Session() as sess:
    
    # Run the initializer
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(num_epochs):
        indices = np.random.permutation(samples);
        
        # Compute the loss across all the batches
        total_loss = 0;
        for i in range(num_batches):
            x_train, y_train = next_batch(indices, i);
            [loss, train] = sess.run([loss_op, train_op], feed_dict={X: x_train, Y: y_train});            
            total_loss += loss;
        
        loss_arr.append(total_loss / num_batches);
        print(epoch, total_loss / num_batches);

12
0 1104.1406161
1 992.613292694
2 963.476533254
3 896.882708867
4 858.110800425
5 836.51279513
6 822.502759933
7 815.535288493
8 810.30205663
9 798.202236176
10 795.988983154
11 796.320659637
12 784.355153402
13 780.578964233
14 773.817652384
15 767.65655454
16 762.880091985
17 765.170710882
18 763.368068695
19 759.173635483
20 753.837003708
21 750.426827749
22 747.753831863
23 743.867211024
24 741.519345601
25 739.848567963
26 736.294485728
27 734.93744278
28 739.295982361
29 734.517702738
30 746.778478622
31 737.499676387
32 741.798293432
33 735.20387586
34 728.17932574
35 725.868217468
36 727.015832265
37 726.189908346
38 721.746503194
39 719.592126211
40 716.827125549
41 714.216209412
42 715.06918335
43 714.249135335
44 713.811274211
45 719.670748393
46 707.069187164
47 702.964064916
48 703.750979741
49 700.27777799
50 699.88539505
51 692.908416748
52 691.464989344
53 691.045251211
54 692.434778214
55 694.218255361
56 683.545743306
57 681.727420171
58 680.947147369
59 682.9848155

In [ ]:
plt.plot(loss_arr);